
## Project:  Allstate Claims Severity
#### Author:   Joseph Downs, James Peng, Megan Pera, Diana Rodenberger 
#### Purpose:  Evaluate multiple regression models to predict cost and severity of claims for AllState
#### Created:  10/29/2016

### Team name in Kaggle: UCB_207_1

## Link to Leaderboard
https://www.kaggle.com/c/allstate-claims-severity/leaderboard


### Topics included

#### 1) Definition of global variables
#### 2) Preprocessing train set: create dummy variables, split dataset into train and dev
#### 3) Function to estimate best parameters for classifiers
#### 4) Data Exploration of Continous Variables
#### 5) Model 1: Linear Regression with dummy variables using RIDGE
#### 6) Model 2: PCA and Regression
#### 7) Model 3: Run regression models with Lasso
#### 8) Model 4: Run regression models with ElasticNet
#### 9) Model 5: Run regression models with DecisionTreeRegressor
#### 10) Model 6: Run regression models with XGBoost





In [1]:
%matplotlib inline
import unittest

# General libraries.
import re, os, sys
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
#from sklearn.feature_extraction import 
from sklearn import preprocessing

from sklearn.utils import shuffle

from sklearn.metrics import mean_absolute_error

from sklearn import linear_model

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from pandas.tools.plotting import scatter_matrix

from sklearn.linear_model import LinearRegression

# regularization methos
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.grid_search import GridSearchCV

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Global variables

In [2]:
##GLOBAL VARIABLES

DUMMY_COLS=[]  #list of dummy variables
CONTINOUS_COLS=[]  #list of continous variables
#R_CLS          #Regression classifier defined in multiple sections of the code

## Preprocessing train set

 1) Extract data and load into dataframe
 
 2) Create dummy variables for categorical variables
 
 3) Transform the variable 'loss'
 
 4) Create training datasets and development datasets.

In [3]:
df_data = pd.read_csv('./data_in/train.csv')
df_data.shape

df_test = pd.read_csv('./data_in/test.csv')
df_test.shape

(125546, 131)

In [4]:
features = df_data.columns
cat_cols = [feat for feat in features if 'cat' in feat]
cont_cols = [feat for feat in features if 'cont' in feat]

#convert categorical variables into dummy variables
df_dummy_vars=pd.get_dummies(
                pd.concat([df_data[cat_cols], df_test[cat_cols]], axis=0)
                )

In [ ]:
df_dummy_vars.shape

In [ ]:
df_dummy_vars[:df_data.shape[0]].shape

In [5]:
#split data frame into categorical variables, continous variables (including all 'contx' 
#variables and 'loss') and ids

features = df_data.columns
cat_cols = [feat for feat in features if 'cat' in feat]
cont_cols = [feat for feat in features if 'cont' in feat]


df_cont_vars =  df_data[cont_cols]
        
    
#convert categorical variables into dummy variables
df_dummy_vars=pd.get_dummies(
                pd.concat([df_data[cat_cols], df_test[cat_cols]], axis=0)
                )

#set global vars for later use
DUMMY_COLS=df_dummy_vars.columns
CONTINOUS_COLS=cont_cols


#create dataset with predictors
X = pd.concat([df_dummy_vars[:df_data.shape[0]], df_cont_vars], axis=1)
#create dataset with dependent variable
y = np.log10(df_data.loss)
#create additioanl dataset with claim ids
Id = df_data.id


In [6]:
X_train, X_dev, y_train, y_dev, id_train, id_dev = train_test_split( X, y, Id, test_size=0.33, random_state=1)

In [7]:
# Set variables to hold dev and training data
# X_dev, y_dev, dev_id = X[168318:], y[168318:], id[168318:]
# X_train, y_train, train_id = X[:168318], y[:168318], id[:168318]
X_train_mini, y_train_mini, id_train_mini = X_train[:40000], y_train[:40000], id_train[:40000]

print('X_train shape: ',X_train.shape)
print('y_train shape: ',y_train.shape)
print('X_dev shape: ',X_dev.shape)
print('y_dev shape: ',y_dev.shape)
print('X_train shape: ',X_train_mini.shape)
print('y_train shape: ',y_train_mini.shape)

X_train shape:  (126173, 1190)
y_train shape:  (126173,)
X_dev shape:  (62145, 1190)
y_dev shape:  (62145,)
X_train shape:  (40000, 1190)
y_train shape:  (40000,)


In [8]:
X_train.head(2)

,cat1_A,cat1_B,cat2_A,cat2_B,cat3_A,cat3_B,cat4_A,cat4_B,cat5_A,cat5_B,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
18739,1,0,1,0,1,0,1,0,0,1,...,0.380560,0.608838,0.546949,0.45289,0.42289,0.55533,0.511698,0.557380,0.642600,0.221177
155434,1,0,1,0,1,0,1,0,0,1,...,0.568839,0.439206,0.407462,0.45883,0.46853,0.52221,0.441763,0.443374,0.324464,0.340194


### Visualizing greatest and lowest (bottom and top 10%) risk in a 2D feature space

In [ ]:
# Initialize PCA, k = 2
pca2 = PCA(n_components = 2)
# Fit model and apply the dimensionality reduction to train_data
values = pca2.fit_transform(X_train) # returns a (126173, 2) array

# Pull out values that are high and low risk 
h,l =[],[] # initialize empty list
risk = np.asarray(y_train)

for k in range(0,126173): 
    if risk[k] >= 3.806554: 
        h.append(k) # append index to h if in top 10% of risk
    elif risk[k] <= 2.888932:
        l.append(k) # append index to l if in bottom 10% of risk

# Split into high (above mean) and low (below mean) categories
high = np.asarray([values[i] for i in h]) 
low = np.asarray([values[i] for i in l])
h_toplot = np.transpose(high)
l_toplot = np.transpose(low)

# Plot arrays, red is high and blue is low
fig = plt.subplots(figsize=(10.0,8.0))
high, = plt.plot(h_toplot[0],h_toplot[1],'ro', label='High')
low, = plt.plot(l_toplot[0],l_toplot[1],'bo',label='Low')
plt.legend(handles=[high, low])
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('2D Feature Space for Allstate Risk Classification')
plt.show()

### Data Exploration of Continous Variables

1) Check the distributions 

2) Check for autocorrelation between variables

3) Check variances to see if noramalization is necessary

In [ ]:
#function to plot histograms of continous variables
def PlotHisContinousVars(df):

    plt.subplots(5,3, figsize=(10,15))
    jj = 0
    for column_name in df.columns:
        jj+=1
        plt.subplot(5,3,jj).hist(df[column_name],normed=False, bins=60)
        plt.subplot(5,3,jj).set(title=column_name)
    plt.show()
 
PlotHisContinousVars(df_cont_vars)

Plot dependent variable

In [ ]:
plt.cla()
plt.subplot(1,1,1).hist(y_train,normed=True, bins=60)
plt.subplot(1,1,1).set(title='loss')
plt.show()

### Investigating autocorrelation

Visualizing a scatter plot matrix for continuous variables to investigate autocorrelation.

In [ ]:
# Building the scatterplot matrix using the training data
scatter_matrix(df_cont_vars, alpha=0.02, figsize=(14, 14), diagonal='kde')
plt.show()

### Investigating the variance of continuous variables

The plot below shows the variances for each of the 14 continuous variables in this dataset. All of the variances are less than 0.05, and the range of the variance is low (less than 0.02). This indicates that these variables have likely already been normalized. Since there is no great differences in variance between these variables, we will do no normalization ourselves.

In [ ]:
print('Range of variance:',df_cont_vars.var().max() - df_cont_vars.var().min())
df_cont_vars.var().plot(kind='bar', alpha =0.4)
plt.show()

### Function to estimate best parameters for classifiers

Warning: The call to this function can take a long time.

In [9]:
def EvaluateHyperParams(cls, params, X_data, y_data):
    
    pipeline = Pipeline([
     ('clf', cls),
    ])
    
    grid_search = GridSearchCV(pipeline, params, scoring = 'neg_mean_absolute_error')
    grid_search.fit(X_data, y_data)
    
    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    return best_parameters

### Baseline scores

The following are the baseline scores submitted for the competition

mean_absolute_error on training data: [-0.19872471 -0.19753498 -0.19821047]

mean_absolute_error on test data 0.198086964004104


#### Function to estimate MAE for specified regression classifier (Ridge, Lasso, etc)

In [10]:
def EstimateMAEForRegression(cls, X_train,y_train,X_test, y_test):
    
    # Train the model using the training sets
    cls.fit(X_train, y_train)

    # predict dev set
    preds = cls.predict(X_test)
   
    s = cross_val_score(cls, X_train, y_train, scoring='neg_mean_absolute_error')
    
    print('mean_absolute_error on training data: {0}'.format(s))

    mae = mean_absolute_error(y_test, preds)
    print('mean_absolute_error on dev data {0}'.format(mae))

### Model 1: Linear Regression with dummy variables using RIDGE

We will try 3 different values of alpha

TODO: Find optimal values of alpha combined with other hyper-parameters

Find optimal value of alpha

In [ ]:
parameters = {
    'clf__alpha': [0.000001, 0.001, 0.01, 0.1 ],
    'clf__normalize': [True]
    }
best_parameters=EvaluateHyperParams(cls=linear_model.Ridge(),params=parameters,X_data=X_train, y_data=y_train)

#### alpha =0.00001

In [ ]:
pd.options.mode.chained_assignment = None

alpha=0.00001

r_cls=linear_model.Ridge(alpha=alpha, normalize=True)

print("Regression - Ridge with alpha : ", alpha)

EstimateMAEForRegression(cls=r_cls,X_train=X_train,y_train=y_train,X_test=X_dev, y_test=y_dev)

Regression - Ridge with alpha :  1e-05


#### alpha =0.001

In [ ]:
alpha=0.001

r_cls=linear_model.Ridge(alpha=alpha, normalize=True)

print("Regression - Ridge with alpha : ", alpha)

EstimateMAEForRegression(cls=r_cls,X_train=X_train,y_train=y_train,X_test=X_dev, y_test=y_dev)

#### alpha =0.1

In [ ]:
alpha=0.1

r_cls=linear_model.Ridge(alpha=alpha, normalize=True)

print("Regression - Ridge with alpha : ", alpha)

EstimateMAEForRegression(cls=r_cls,X_train=X_train,y_train=y_train,X_test=X_dev, y_test=y_dev)

### Model 2: PCA and Regression

Create a model for regression that uses PCA components for dummy variables + continous variables. Here are the steps to run this model:

1) Create a dataset with PCA components of dummy variables. 

2) Create a dataset with just continous variables. 

3) Create a combined dataset using datasets from step 1 and 2.

4) Do steps 1-3 for training data and dev data.

5) Run regression.

### Pre-processing: Find the number of components that explains most of the variance.

In [ ]:


def ExploreNumPCAComponents(n_pca_comp):

    pca = PCA(n_components=n_pca_comp)
    pca.fit(X_train)
    cum_vars=[]
    components=[]
    cumulative_var=0
    for ii in range(len(pca.explained_variance_ratio_)):
        cumulative_var+=pca.explained_variance_ratio_[ii]
        cum_vars.append(cumulative_var)
        components.append(ii+1)
        
    plt.plot(components, cum_vars)
    
    plt.show()
    return cum_vars

cum_vars=ExploreNumPCAComponents(200)



In [ ]:
print('Total variance explained by first PCA component: %0.6f \n' % cum_vars[0])
print('Total variance explained by 8 PCA components: %0.6f \n' % cum_vars[8])
print('Total variance explained by 50 PCA components: %0.6f \n' % cum_vars[49])
print('Total variance explained by 100 PCA components: %0.6f \n' % cum_vars[99])
print('Total variance explained by 215 PCA components: %0.6f \n' % cum_vars[214])

##### Total variance explained by 215 components is 95.93%. 

### Create Training set and Development Set with PCA components + Continous Variables

Function to create a dataframe with PCA components. 

In [ ]:
def PCAComponentsToDataFrame(PCAComp, index_list):
    #### Purpose: Transform PCA components to DataFrame
    dic_comp={}
    ii=0
    for ii in range(PCAComp.shape[1]):
        key= 'comp' + str(ii+1)
        dic_comp[key]=PCAComp[:,ii]

    #create dataframes with pca components
    return pd.DataFrame(dic_comp, index=index_list)
    

##### Create a new training set and dev set with pca components

Try 215 components because that explains the most variance

In [ ]:
n_pca_comp=215

#Run PCA with with 215 components
pca = PCA(n_components=n_pca_comp)
train_data_pca = pca.fit_transform(X_train)
dev_data_pca = pca.transform(X_dev)

#### Transform PCA components to DataFrame, preserve the index of the original datasets.
X_train_pca=PCAComponentsToDataFrame(train_data_pca,list(id_train.index))
X_dev_pca=PCAComponentsToDataFrame(dev_data_pca,list(id_dev.index))


In [ ]:
#checks
print(X_train.shape)
print(X_train_pca.shape)
print(X_dev_pca.shape)

### Run regression models with 215 PCA components

In [ ]:
parameters = {
    'clf__alpha': (0.00001, .001, 0.1, 0.8),
    'clf__solver': ('auto', 'sag'),
    }
best_parameters=EvaluateHyperParams(cls=linear_model.Ridge(),params=parameters,X_data=X_train_pca, y_data=y_train)

In [ ]:
alpha=0.00001

r_cls=linear_model.Ridge(alpha=alpha, normalize=True)

print("Regression with PCA components and continous variables -Ridge with alpha : ", alpha)

EstimateMAEForRegression(cls=r_cls,X_train=X_train_pca,y_train=y_train,X_test=X_dev_pca, y_test=y_dev)

In [ ]:
alpha=0.1

r_cls=linear_model.Ridge(alpha=alpha, normalize=True)

print("Regression with PCA components and continous variables -Ridge with alpha : ", alpha)

EstimateMAEForRegression(cls=r_cls,X_train=X_train_pca,y_train=y_train,X_test=X_dev_pca, y_test=y_dev)

In [ ]:
alpha=0.0000001

r_cls=linear_model.Ridge(alpha=alpha, normalize=True)

print("Regression with PCA components and continous variables -Ridge with alpha : ", alpha)

EstimateMAEForRegression(cls=r_cls,X_train=X_train_pca,y_train=y_train,X_test=X_dev_pca, y_test=y_dev)

The following function features columns with the larges coefficients.

In [ ]:
def SelectFeaturesLargestCoeff(cls, X):
    #select columns with top coe
    col_names=X.columns

    sorted_index=np.argsort(cls.coef_)
    best_coeff_vars=[]
    for ii in range(15):
        best_coeff_vars.append(col_names[sorted_index[(len(sorted_index)-ii-1)]])
    return best_coeff_vars


Use features with the largest coefficients

In [ ]:
best_coeff_vars=[]
best_coeff_vars=SelectFeaturesLargestCoeff(r_cls,X_train_pca)
df_X_train_pca_best=X_train_pca[best_coeff_vars]
df_X_dev_pca_best=X_dev_pca[best_coeff_vars]

In [ ]:
parameters = {
    'clf__alpha': (0.00001, .001, 0.1, 0.8),
     }
best_parameters=EvaluateHyperParams(cls=linear_model.Ridge(),params=parameters,X_data=df_X_train_pca_best, y_data=y_train)

In [ ]:
alpha=0.8

r_cls=linear_model.Ridge(alpha=alpha, normalize=True)

print("Regression using features with largest coefficients (PCA comp + cont var) -Ridge with alpha : ", alpha)

EstimateMAEForRegression(cls=r_cls,X_train=df_X_train_pca_best,y_train=y_train,X_test=df_X_dev_pca_best, y_test=y_dev)



### Model 3: Run regression models with Lasso

In [ ]:
def EvaluateHyperParams(cls, params, X_data, y_data):    
    pipeline = Pipeline([
     ('clf', cls),
    ])

    grid_search = GridSearchCV(pipeline, params)
    grid_search.fit(X_data, y_data)
    
    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    return best_parameters

In [ ]:
parameters = {
    'clf__alpha': (0.00001, .001, 0.1, 0.8),
     }

best_parameters=EvaluateHyperParams(cls=linear_model.Lasso(),params=parameters,X_data=X_train_mini, y_data=y_train_mini)

In [ ]:
alpha=0.00001

r_cls=linear_model.Lasso(alpha=alpha, normalize=True)

print("Regression using PCA comp + cont var -Lasso with alpha : ", alpha)

EstimateMAEForRegression(cls=r_cls,X_train=X_train_mini,y_train=y_train_mini,X_test=X_dev, y_test=y_dev)

### Model 4: Run regression models with ElasticNet

In [ ]:
parameters = {
    'clf__alpha': (0.00001, .001, 0.1, 0.8),
     }

best_parameters=EvaluateHyperParams(cls=linear_model.ElasticNet(),params=parameters,X_data=X_train_pca, y_data=y_train)

In [ ]:
alpha=0.1

r_cls=linear_model.ElasticNet(alpha=alpha, normalize=True)

print("Regression -ElasticNet with alpha : ", alpha)

EstimateMAEForRegression(cls=r_cls,X_train=X_train,y_train=y_train,X_test=X_dev, y_test=y_dev)

### Model 5: Run regression models with DecisionTreeRegressor

In [ ]:
max_depth=5

r_cls=DecisionTreeRegressor(max_depth=max_depth)

print("Regression -DecisionTreeRegressor with max_depth : ", max_depth)

EstimateMAEForRegression(cls=r_cls,X_train=X_train,y_train=y_train,X_test=X_dev, y_test=y_dev)


In [ ]:
max_depth=15

r_cls=DecisionTreeRegressor(max_depth=max_depth)

print("Regression -DecisionTreeRegressor with max_depth : ", max_depth)

EstimateMAEForRegression(cls=r_cls,X_train=X_train,y_train=y_train,X_test=X_dev, y_test=y_dev)


### Model 6: using XGBoost

In [ ]:
# Import xgboost python module
import xgboost as xgb

In [ ]:
# fitting model on training data
xgbr = xgb.XGBRegressor(max_depth=6, n_estimators=500, learning_rate=0.1, subsample=0.8, 
                        colsample_bytree=0.4, min_child_weight = 3,  seed=7)

EstimateMAEForRegression(cls=xgbr,X_train=X_train,y_train=y_train,X_test=X_dev, y_test=y_dev)

### Model 7: AdaBoost Regression

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

abr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=5))
EstimateMAEForRegression(cls=abr,X_train=X_train_mini,y_train=y_train_mini,X_test=X_dev, y_test=y_dev)

### Predict the Test Dataset

 1) Extract data and load into dataframe
 
 2) Create dummy variables for categorical variables
 
 3) Predict loss based on 'best' model previously found : Ridge with alpha= 0.00001


In [ ]:
# = pd.read_csv('./data_in/test.csv')

In [ ]:
#test_cat_cols=[]
#test_cont_cols=[]
#for c in df_test.columns:
    #if c.find('cat') >=0: # -1: substring not found, >=0, starting index
        #test_cat_cols.append(c)
    #elif c.find('cont')>=0:
        #test_cont_cols.append(c)

#test_df_cont_vars = df_test[test_cont_cols]
        
#convert categorical variables into dummy variables
#test_df_dummy_vars=pd.get_dummies(df_test[test_cat_cols])

#create dataset with predictors
#X_test = pd.concat([test_df_dummy_vars, test_df_cont_vars], axis=1)
#id_test = df_test.id


In [ ]:

#print('num columns in test: ', X_test.shape[1])
#print('num columns in train: ', X_train.shape[1])

## Important note:

When transforming categorical variables into dummy variables, we get a different set of variables for the training data and the test data. The train data and the test data have different categories for some categorical variables; thus, when we tranform categorical variables to dummy variables then a different set of dummy variables for the train and test data. 

See file 'Allstate - Create train and test datasets with dummy vars' to fix this problem
